In [3]:
import numpy as np

In [4]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=xszZMpcfq2VDyAqMuCAG6oeKsYs9m7XAXfD2xx_JREA&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhKEUrqrQykDGDcpLXfZsIB8oJ1nlt7m1QVkP65EGop8-cT4djd83w

Successfully saved authorization token.


In [5]:
rain_daily_CHIRPS = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
lst_daily_MODIS = ee.ImageCollection("MODIS/006/MOD11A1")
countries = ee.FeatureCollection('FAO/GAUL/2015/level0').select('ADM0_NAME')

In [6]:
#Land ST dates
# Initial date of interest (inclusive).
i_date = '2020-01-01'
# Final date of interest (exclusive).
f_date = '2020-12-30'

#precipitation dates

# Filter the feature collection to subset France.
# roi = countries.filter(ee.Filter.eq('ADM0_NAME', 'India'))
india = countries.filter(ee.Filter.eq('ADM0_NAME', 'India'))

# Selection of appropriate bands and dates for LST.
rain_daily = rain_daily_CHIRPS.filterDate(i_date, f_date).select('precipitation')
lst_daily = lst_daily_MODIS.filterDate(i_date, f_date).select('LST_Day_1km')

In [7]:
print(rain_daily.size().getInfo())

364


In [8]:
#Bangalore
u_lon = 77.5946
u_lat = 12.9716
u_poi = ee.Geometry.Point(u_lon, u_lat)

In [9]:
roi = u_poi.buffer(1900000)

In [10]:
rain_img = rain_daily.mean()

In [11]:
lst_img = lst_daily.mean()
# Adjust for scale factor.
lst_img = lst_img.multiply(0.02)
# Convert Kelvin to Celsius.
lst_img = lst_img.add(-273.15)

In [12]:
from IPython.display import Image as IPyImage

In [13]:
lst_india = lst_img.clip(india)
rain_india = rain_img.clip(india)

In [14]:
lstVisParams = {
  'min': 1,
  'max': 50,
  'region':roi,
  'dimensions' : 512,
  'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ],
};

Satellite images often present missing data due to atmospheric aerosol, clouds or other atmospheric conditions. 
https://modis.gsfc.nasa.gov/sci_team/pubs/abstract.php?id=07273

In [15]:
url = lst_india.getThumbUrl(lstVisParams)
print(url)

# Display the thumbnail land surface temperature in India.
print('\nPlease wait while the thumbnail loads, it may take a moment...')
IPyImage(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/a43d7513e7fd5b91fd286025812cff32-a9bb5334969d03ff5278d1630196d580:getPixels

Please wait while the thumbnail loads, it may take a moment...


In [16]:
precipitationVisParams = {
  'min': 1.0,
  'max': 17.0,
  'region': roi,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000'],
};

In [17]:
url = rain_india.getThumbUrl(precipitationVisParams)
print(url)

# Display the thumbnail land surface temperature in India.
print('\nPlease wait while the thumbnail loads, it may take a moment...')
IPyImage(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/22b619376b44e5b1956636cc34483b8e-b7316e26cc0d9a4ff015a45f95175429:getPixels

Please wait while the thumbnail loads, it may take a moment...


In [18]:
import urllib.request
from PIL import Image
urllib.request.urlretrieve(url, 'file_name')
lst_img = Image.open('file_name')

#FOLIUM

In [20]:
import folium

# Define the center of our map.
lat, lon = u_lat, u_lon

my_map = folium.Map(location=[lat, lon], zoom_start=10)
my_map

In [21]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [22]:
# Create a map.
my_map = folium.Map(location=[u_lat, u_lon], zoom_start=7)

# Add the land cover to the map object.
my_map.add_ee_layer(lst_india, lstVisParams, 'Land Surface Temps Cover')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [23]:
# Create a map.
my_map = folium.Map(location=[u_lat, u_lon], zoom_start=7)

# Add the land cover to the map object.
my_map.add_ee_layer(rain_india, precipitationVisParams, 'Rain Cover')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)